In [2]:
from surprise import Dataset, Reader, SVD,KNNWithMeans

In [3]:
import pandas as pd

In [4]:
from surprise.model_selection import cross_validate

In [5]:
ratings = pd.read_csv("E:/Internship/data/ratings_cleaned_v1.csv",sep="\t")

In [6]:
reader = Reader()

data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

In [ ]:
svd = SVD()

In [7]:
train_set = data.build_full_trainset()

In [8]:
Knn_model = KNNWithMeans(
    sim_options={'name':'cosine',
                 'user_based':False}
    ).fit(train_set).compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [14]:
k =20

In [77]:
test_subject = 2

test_subject_iid = train_set.to_inner_uid(test_subject)

In [78]:
test_subject_ratings = train_set.ur[test_subject_iid]

In [11]:
import heapq

In [79]:
test_subject_ratings

[(0, 1.0),
 (1, 4.5),
 (2, 5.0),
 (3, 5.0),
 (4, 4.0),
 (5, 4.5),
 (6, 4.0),
 (7, 4.0),
 (8, 3.5),
 (9, 4.0),
 (10, 5.0)]

In [80]:
k_neighbours = heapq.nlargest(k, test_subject_ratings, key=lambda t:t[1])

In [81]:
k_neighbours

[(2, 5.0),
 (3, 5.0),
 (10, 5.0),
 (1, 4.5),
 (5, 4.5),
 (4, 4.0),
 (6, 4.0),
 (7, 4.0),
 (9, 4.0),
 (8, 3.5),
 (0, 1.0)]

In [17]:
from collections import defaultdict

In [82]:
candidates = defaultdict(float)

In [83]:
for itemID, rating in k_neighbours:
    try:
        similarities = Knn_model[itemID]
        for innerID, score in enumerate(similarities):
            candidates[innerID] += score * (rating/10)
    except:
        continue



In [84]:
movies = pd.read_csv("E:/Internship/data/movies_final.csv",sep="\t")

In [85]:
movies

,id,original_title,title,new_title,imdb_id,budget,revenue,year,runtime,release_date,age_rating,vote_average,vote_count,popularity,critical_score
0,862,Toy Story,Toy Story,Toy Story (1995),tt0114709,30000000,373554033.0,1995,81.0,1995-10-30,G,7.7,5415.0,21.946943,2353.324641
1,8844,Jumanji,Jumanji,Jumanji (1995),tt0113497,65000000,262797249.0,1995,104.0,1995-12-15,PG,6.9,2413.0,17.015539,778.136085
2,15602,Grumpier Old Men,Grumpier Old Men,Grumpier Old Men (1995),tt0113228,0,0.0,1995,101.0,1995-12-22,PG-13,6.5,92.0,11.712900,26.278206
3,31357,Waiting to Exhale,Waiting to Exhale,Waiting to Exhale (1995),tt0114885,16000000,81452156.0,1995,127.0,1995-12-22,R,6.1,34.0,3.859495,8.715714
4,11862,Father of the Bride Part II,Father of the Bride Part II,Father of the Bride Part II (1995),tt0113041,0,76578911.0,1995,106.0,1995-02-10,PG,5.7,173.0,8.387519,40.223204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45394,439050,رگ خواب,Subdue,Subdue (2017),tt6209470,0,0.0,2017,90.0,2017-06-14,Not Rated,4.0,1.0,0.072051,0.166639
45395,111109,Siglo ng Pagluluwal,Century of Birthing,Century of Birthing (2011),tt2028550,0,0.0,2011,360.0,2011-11-17,Not Rated,9.0,3.0,0.178241,2.997003
45396,67758,Betrayal,Betrayal,Betrayal (2003),tt0303758,0,0.0,2003,90.0,2003-08-01,R,3.8,6.0,0.903007,0.967586
45397,227506,Satana likuyushchiy,Satan Triumphant,Satan Triumphant (1917),tt0008536,0,0.0,1917,87.0,1917-10-21,Not Rated,0.0,0.0,0.003503,0.000000


In [86]:
def getMoviename(movieId):
    if int(movieId) in movies['id'].values:
        return movies[movies['id'] == int(movieId)]['new_title'].values[0]

    else:
        print("Movie Not Found")

In [87]:
getMoviename(16)

'Dancer in the Dark (2000)'

In [88]:
from operator import itemgetter

In [89]:
watched = {}

for itemId, rating in train_set.ur[test_subject_iid]:
    watched[itemId]=1

recommendations = []

for itemId, rating_sum in sorted(candidates.items(), key=itemgetter(1),reverse=True):
    if not itemId in watched:
        recommendations.append(getMoviename(train_set.to_raw_iid(itemId)))
        if len(recommendations) >= 10:
            break

In [90]:
recommendations

['Mad at the Moon (1992)',
 'The House of Smiles (1991)',
 'Malarek (1988)',
 'Melody for a Street Organ (2009)',
 'Taken (2002)',
 'Jesus liebt mich (2012)',
 'Mr. Blue Sky: The Story of Jeff Lynne & ELO (2012)',
 'The Crush (1967)',
 'Harvey (1996)',
 "St. Martin's Lane (1938)"]

In [91]:
import joblib

In [92]:
joblib.dump(Knn_model, "knn_sim.joblib")

['knn_sim.joblib']

In [16]:
cross_validate(svd, data, measures=["RMSE","MAE"],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8359  0.8347  0.8342  0.8343  0.8335  0.8345  0.0008  
MAE (testset)     0.6376  0.6361  0.6357  0.6358  0.6354  0.6362  0.0008  
Fit time          124.06  129.36  119.55  122.13  114.25  121.87  4.99    
Test time         34.99   30.88   33.09   27.95   24.69   30.32   3.67    


{'test_rmse': array([0.83589183, 0.83466691, 0.83421856, 0.83431854, 0.83348297]),
 'test_mae': array([0.6376367 , 0.63614984, 0.63574123, 0.63584994, 0.63542705]),
 'fit_time': (124.05709910392761,
  129.36240482330322,
  119.54818558692932,
  122.13179969787598,
  114.24895453453064),
 'test_time': (34.99195742607117,
  30.88482093811035,
  33.09029459953308,
  27.9517023563385,
  24.6851589679718)}

In [25]:
from surprise import PredictionImpossible

In [27]:
svd.predict(uid=2,iid=6)

Prediction(uid=2, iid=6, r_ui=None, est=3.661902093457013, details={'was_impossible': False})